<a href="https://colab.research.google.com/github/subornaa/Data-Analytics-Tutorials/blob/main/Data_Wrangling_Tutorials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">

# Data Wrangling in Python
</div>

![prf_image](https://opendata.nfis.org/mapserver/PRF_Layout.jpg)

This notebook provides python code to analyze the Petawawa Research Forest (PRF) inventory and ancillary datasets.

All data is sourced from the following website:

https://opendata.nfis.org/mapserver/PRF.html

Please consult this page to get the most up to date version of the PRF data.

# Dataset Background

The Petawawa Research Forest (PRF) is the oldest research forest in Canada, dating back to 1918. As such, the area has a rich historical record of data, and has recently become a remote sensing "supersite".

You can read more about the PRF as a supersite in [White et al. (2019)](https://doi.org/10.5558/tfc2019-024).


**References:**

White, Joanne C., et al. "The Petawawa Research Forest: Establishment of a remote sensing supersite." The Forestry Chronicle 95.3 (2019): 149-156.



# Install and load required packages

In [ ]:
import os
import shutil
import pandas as pd
import geopandas as gpd

# Download data

In [ ]:
# Download the data if it does not yet exist
if not os.path.exists("data"):
  !gdown 1UDKAdXW0h6JSf7k31PZ-srrQ3487l9e2
  !unzip prf_data.zip -d data/
  os.remove("prf_data.zip")
else:
  print("Data has already been downloaded.")

!ls data/

Data has already been downloaded.
plots.gpkg  trees.csv


# Data Wrangling

In [ ]:
trees_df = pd.read_csv("data/trees.csv")
trees_df.head()


,tree_spec,PlotName,TreeID,TreeSpec,Origin,Status,DBH,CrownClass,QualityClass,DecayClass,...,BA_all,TPH_all,codom,domht,ht_meas,stems,mvol,tvol,biomass,size
0,1,PRF001,24.0,1,P,D,10.1,NaN,NaN,1.0,...,33.601655,2688,N,12.223077,NaN,16,0.000000,0.708735,393.396400,Poles
1,1,PRF001,46.0,1,P,D,9.9,NaN,NaN,2.0,...,33.601655,2688,N,12.223077,NaN,16,0.000000,0.673254,375.305379,Poles
2,2,PRF001,20.0,2,N,L,67.5,D,A,NaN,...,33.601655,2688,Y,33.433333,33.9,16,77.327438,79.482658,39691.639950,Large
3,2,PRF001,50.0,2,N,L,57.9,D,U,NaN,...,33.601655,2688,Y,33.433333,NaN,16,56.444281,58.117292,28251.255888,Large
4,1,PRF001,10.0,1,N,L,55.9,D,A,NaN,...,33.601655,2688,Y,33.433333,33.0,16,48.008649,49.833743,24501.838779,Large
